In [1]:
import SimpleITK as sitk
ref_img = r'/home/songzhiyun/project/BoneTumorSeg/data/version2/83/83-dicom.nii.gz'
ref_img = sitk.ReadImage(ref_img)
spacing = ref_img.GetSpacing()
origin = ref_img.GetOrigin()
direction = ref_img.GetDirection()

In [ ]:
resampled_img = resample_img(ref_img, spacing, new_spacing=[spacing[0], spacing[1], spacing[2]*4])


In [18]:
lr_data = sitk.GetArrayFromImage(ref_img)
lr_data = lr_data[::4,...]
lr_img = sitk.GetImageFromArray(lr_data)
lr_img.SetSpacing((spacing[0], spacing[1], spacing[2]*4))
lr_img.SetOrigin(origin)
lr_img.SetDirection(direction)
sitk.WriteImage(lr_img, '../all_results/segsr_test/result_baseline/lr_img.nii.gz')

In [19]:
sr_img = resample_img(lr_img, spacing)
sitk.WriteImage(sr_img, '../all_results/segsr_test/result_baseline/sr_img.nii.gz')

In [6]:
import numpy as np
def resample_img(itk_image, out_spacing=[2.0, 2.0, 2.0], is_label=False):
    # resample images to 2mm spacing with simple itk

    original_spacing = itk_image.GetSpacing()
    original_size = itk_image.GetSize()

    out_size = [
        int(np.round(original_size[0] * (original_spacing[0] / out_spacing[0]))),
        int(np.round(original_size[1] * (original_spacing[1] / out_spacing[1]))),
        int(np.round(original_size[2] * (original_spacing[2] / out_spacing[2])))]

    resample = sitk.ResampleImageFilter()
    resample.SetOutputSpacing(out_spacing)
    resample.SetSize(out_size)
    resample.SetOutputDirection(itk_image.GetDirection())
    resample.SetOutputOrigin(itk_image.GetOrigin())
    resample.SetTransform(sitk.Transform())
    resample.SetDefaultPixelValue(itk_image.GetPixelIDValue())

    if not is_label:
        resample.SetInterpolator(sitk.sitkBSpline)

    return resample.Execute(itk_image)

In [7]:
out_seg_img = r'../all_results/segsr_test_addflip/trilinear/pred_lr.nii.gz'
out_seg_img = sitk.ReadImage(out_seg_img)
out_seg_img.SetSpacing((spacing[0], spacing[1], spacing[2]*4))
out_seg_img.SetOrigin(origin)
out_seg_img.SetDirection(direction)
sitk.WriteImage(out_seg_img, '../all_results/segsr_test_addflip/trilinear/pred_lr.nii.gz')

In [16]:
out_seg_img_sr = resample_img(out_seg_img, out_spacing=spacing, is_label=True)
out_seg_img_sr_data = sitk.GetArrayFromImage(out_seg_img_sr)
out_seg_img_sr_data = np.round(out_seg_img_sr_data)
out_seg_img_sr = sitk.GetImageFromArray(out_seg_img_sr_data)
out_seg_img_sr.SetSpacing(spacing)
out_seg_img_sr.SetOrigin(origin)
out_seg_img_sr.SetDirection(direction)
sitk.WriteImage(out_seg_img_sr, '../all_results/segsr_test/result_nnunet/pred_sr.nii.gz')

In [8]:
out_seg_img = r'../all_results/segsr_test_addflip/trilinear/pred_hr.nii.gz'
out_seg_img = sitk.ReadImage(out_seg_img)
out_seg_img.SetSpacing(spacing)
out_seg_img.SetOrigin(origin)
out_seg_img.SetDirection(direction)
sitk.WriteImage(out_seg_img, '../all_results/segsr_test_addflip/trilinear/pred_hr.nii.gz')

In [2]:
import SimpleITK as sitk
ref_label = r'/home/songzhiyun/project/BoneTumorSeg/data/version2/83/83-seg.nii.gz'
ref_label = sitk.ReadImage(ref_label)
spacing = ref_label.GetSpacing()
origin = ref_label.GetOrigin()
direction = ref_label.GetDirection()

In [7]:
resampled_label = resample_img(ref_label, out_spacing=[spacing[0], spacing[1], spacing[2]*4], is_label=True)

In [9]:
sitk.WriteImage(resampled_label, '/home/songzhiyun/project/BoneTumorSeg/code/smore_seg/smore/all_results/segsr_test/result_baseline/lr_seg.nii.gz')